In [1]:
import os
import sys
from functools import lru_cache
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from svea_expvis.config import Config
from svea_expvis.readers.txt import text_reader
from svea_expvis.bokeh_widgets.map import get_map
from svea_expvis.bokeh_widgets.callbacks import lasso_callback, select_callback, update_colormapper
from svea_expvis.bokeh_widgets.utils import get_columndata_source

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, PreText, Select, Circle
from bokeh.layouts import column, row


DEFAULT_PARAMETERS = ['DOX1', 'OSAT', 'FLU2', 'PHYC', 'phycoerythrin_PHER', 'CDOM_CDOM', 'TUR4', 'PCO2', 'WTEMP', 'PSAL']

cfig = Config(data_file_name='FB_data.txt')

output_notebook()


Loading BokehJS ...

In [2]:
df = text_reader(
    'pandas',
    cfig.data_path,
    sep='\t',
    header=0,
)
df['timestamp'] = df['timestamp'].apply(pd.Timestamp)
df[df==''] = np.nan

datasource = get_columndata_source(df, *DEFAULT_PARAMETERS[:2], lat_col='latitude', lon_col='longitude')

In [3]:
color_mapper = {}
for para in DEFAULT_PARAMETERS:
    color_mapper[para] = LinearColorMapper(
        palette='Turbo256', 
        low=df[para].quantile(.01, interpolation='higher'), 
        high=df[para].quantile(.99, interpolation='lower')
    )

# dummy_cm: not ideal, but it works..
dummy_cm = LinearColorMapper(
        palette='Turbo256', 
        low=df[DEFAULT_PARAMETERS[0]].quantile(.01, interpolation='higher'), 
        high=df[DEFAULT_PARAMETERS[0]].quantile(.99, interpolation='lower')
    )

fig_map = get_map(title='X-parameter')

map_renderer = fig_map.circle(
    x="LONGI", 
    y="LATIT",
    size=3,
    fill_color={'field': DEFAULT_PARAMETERS[0], 'transform': dummy_cm},
    line_color=None,
    alpha=0.5,
    source=datasource,
)
nonselected_circle = Circle(fill_alpha=0.3, line_color='grey')
map_renderer.nonselection_glyph = nonselected_circle

color_bar = ColorBar(
    color_mapper=dummy_cm,
    label_standoff=14,
    location=(0,0),
    title=''
)

fig_map.add_layout(color_bar, 'right')


In [4]:
corr = figure(
    width=400, height=400,
    title='Selected X- and Y values',
    tools='pan,wheel_zoom,box_select,reset,save',
)
corr.xaxis.axis_label = DEFAULT_PARAMETERS[0]
corr.yaxis.axis_label = DEFAULT_PARAMETERS[1]
corr_source = ColumnDataSource(data=dict(x=[], y=[]))
corr.circle(
    x="x", 
    y="y",
    size=3,
    alpha=0.5,
    source=corr_source,
)

GlyphRenderer(id='1101', ...)

In [5]:
x_sel = Select(value=DEFAULT_PARAMETERS[0], options=DEFAULT_PARAMETERS, title='X-parameter')
y_sel = Select(value=DEFAULT_PARAMETERS[1], options=DEFAULT_PARAMETERS, title='Y-parameter')

cm_map_callback = update_colormapper(fig=fig_map, plot=map_renderer, color_mapper=color_mapper, data_source=datasource)

x_sel.js_on_change("value", select_callback(data_source=datasource, axis_objs=[corr.xaxis[0]], axis='x'), cm_map_callback)
y_sel.js_on_change("value", select_callback(data_source=datasource, axis_objs=[corr.yaxis[0]], axis='y'))

In [6]:
selection_change = lasso_callback(
    x_selector=x_sel, y_selector=y_sel, 
    data_source=datasource, corr_source=corr_source
)
datasource.selected.js_on_change('indices', selection_change)

In [7]:
ts1 = figure(width=800, height=250, x_axis_type='datetime', active_drag="xbox_select", title='X-parameter', tools='pan,wheel_zoom,xbox_select,reset,save', toolbar_location="below")
ts1.line('timestamp', 'x', source=datasource)
ts1.circle('timestamp', 'x', size=1, source=datasource, color=None, selection_color="orange")

ts2 = figure(width=800, height=250, x_axis_type='datetime', active_drag="xbox_select", title='Y-parameter', tools='pan,wheel_zoom,xbox_select,reset,save', toolbar_location="below")
ts2.line('timestamp', 'y', source=datasource)
ts2.circle('timestamp', 'y', size=1, source=datasource, color=None, selection_color="orange")

GlyphRenderer(id='1198', ...)

In [8]:
show(row(column(fig_map, ts1, ts2), column(corr, x_sel, y_sel)))